In [69]:
import os
import pandas as pd
from sqlalchemy import create_engine
import altair as alt
import numpy as np
from dotenv import load_dotenv

In [70]:
load_dotenv()

host = os.getenv("MYSQL_HOST", "localhost")
port = os.getenv("MYSQL_PORT", "3306")
user = os.getenv("MYSQL_USER","root")
password = ""
db = os.getenv("MYSQL_DB", "rxnorm")

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)


In [71]:
from sqlalchemy import text

def Searchbar(term):
    # Use text() to safely wrap your query
    sql = text("SELECT * FROM RXNCONSO WHERE STR LIKE :term AND TTY = 'BN'")
    
    with engine.connect() as conn:
        # Pass the parameter as a dictionary
        result = pd.read_sql(sql, conn, params={'term': f'%{term}%'})
    return result

x = Searchbar("Tylenol")
list(x["STR"]) 

['Tylenol',
 'Tylenol PM',
 'Tylenol with Codeine',
 'Tylenol',
 'Tylenol PM',
 'Tylenol with Codeine',
 'Tylenol',
 'Tylenol PM',
 'Tylenol with Codeine']

In [72]:
def Fetch_Drug_Form(name):

    query1 = text("""
        SELECT RXCUI 
        FROM RXNCONSO 
        WHERE STR = :name AND TTY = 'BN'
    """)

    with engine.connect() as conn:
        result1 = pd.read_sql(query1, conn, params={"name": name})

    if result1.empty:
        return pd.DataFrame()

    drug_id = result1["RXCUI"].iloc[0]

    query2 = text("""
        SELECT DISTINCT
            r.RXCUI2 AS Drug_Form_ID, 
            r.RELA AS Relation, 
            c.STR AS Drug_Form
        FROM RXNCONSO c
        JOIN RXNREL r ON c.RXCUI = r.RXCUI2
        WHERE r.RXCUI1 = :drug_id 
          AND c.TTY = 'DP'
    """)

    with engine.connect() as conn:
        result = pd.read_sql(query2, conn, params={"drug_id": drug_id})

    # Create Product_Name first
    result["Product_Name"] = result["Drug_Form"].str.extract(r'\[(.*?)\]')
    result["Product_Name"] = result["Product_Name"].fillna(result["Drug_Form"])

    # Case-insensitive dedup
    result = (
        result.assign(Product_Name_lower=result["Product_Name"].str.lower())
              .drop_duplicates(subset="Product_Name_lower")
              .drop(columns="Product_Name_lower")
    )

    return result

Fetch_Drug_Form("Tylenol PM")

,Drug_Form_ID,Relation,Drug_Form,Product_Name
0,1092378,has_ingredient,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Tylenol PM Extra Strength
1,1092378,has_ingredient,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,Tylenol PM
2,1092378,has_ingredient,ACETAMINOPHEN 500 mg / DIPHENHYDRAMINE HYDROCH...,"Tylenol PM Extra Strength, CVP HEALTH"
3,1092378,has_ingredient,DIPHENHYDRAMINE HYDROCHLORIDE 25 mg / ACETAMIN...,Lil Drug Store Tylenol PM Extra Strength


In [73]:
def Show_Ingredients(ID):
    query = f"""
    SELECT r.RXCUI2 as Ingredient_ID,r.RELA as Relation,c.STR as Ingredient
    from RXNCONSO c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "SCDC"
    GROUP by Ingredient_ID,Relation,Ingredient;
    """
    result1 = pd.read_sql(query, engine)
    return result1

df = Show_Ingredients(1092378)
ingredients = list(df["Ingredient_ID"])
df

,Ingredient_ID,Relation,Ingredient
0,315266,constitutes,acetaminophen 500 MG
1,901813,constitutes,diphenhydramine hydrochloride 25 MG


In [74]:
def Dose_Form(ID):
    query = f"""
    SELECT r.RXCUI2 as Ingredient_ID,r.RELA as Relation,c.STR as Ingredient
    from RXNCONSO c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "DF"
    GROUP by Ingredient_ID,Relation,Ingredient;
    """
    result = pd.read_sql(query, engine)
    DF = result["Ingredient"][0]
    return DF
Dose_Form(209387)

'Oral Tablet'

In [75]:
def get_generic(ID):
    query = f"""
    SELECT r.RXCUI2 as Ingredient_ID,c.STR as Ingredient
    from RXNCONSO c
    JOIN RXNREL r
    ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}" and c.TTY = "SCD"
    GROUP by Ingredient_ID,Ingredient;
    """
    res = pd.read_sql(query, engine)
    return res
    
get_generic(209387)

,Ingredient_ID,Ingredient
0,313782,acetaminophen 325 MG Oral Tablet


In [76]:
def Exact_Drugs(Ing_lst,ID):
    s = ""
    for i,j in enumerate(Ing_lst):
        if i == (len(Ing_lst) - 1):
            s+="r1.RXCUI1 = "+j
        else:
            s+="r1.RXCUI1 = "+j+" or "
            
    query = f"""
    WITH base AS (
        SELECT r2.RXCUI as ID, r2.STR as DP, r1.RXCUI1 as Ingredient_ID
        FROM RXNREL r1
        JOIN RXNCONSO r2
        ON r1.RXCUI2 = r2.RXCUI
        WHERE ({s}) and r2.TTY = "DP"
    ),
    keys_all AS (
        SELECT ID
        FROM base
        GROUP by ID
        HAVING COUNT(DISTINCT Ingredient_ID) = {len(Ing_lst)}
    )
    SELECT b.ID,b.DP
    FROM base b
    JOIN keys_all k
    ON b.ID = k.ID
    WHERE b.Id != {ID}
    GROUP BY b.ID, b.DP
    """
    
    res = pd.read_sql(query, engine)
    
    lst = []
    drp = []
    for j,i in enumerate(res["DP"]): 
        if "[" in i:
            d = i.split("[")
            lst.append(d[-1][:-1])
        else:
            lst.append("Generic")
            drp.append(j)

    res["Product_Name"] = lst
    
    Product = []
    for j,i in enumerate(res["Product_Name"]):
        if i.lower() in Product:
            drp.append(j)
        else:
            Product.append(i.lower())
    res = res.drop(drp)
    res = res.reset_index(drop=True)
    return res
    
df = Exact_Drugs(ingredients,1092378)


In [77]:
def Union_Drugs(Ing_lst,ID):
    s = ""
    for i,j in enumerate(Ing_lst):
        if i == (len(Ing_lst) - 1):
            s+="r1.RXCUI1 = "+j
        else:
            s+="r1.RXCUI1 = "+j+" or "
            
    query = f"""
    WITH base AS (
        SELECT r2.RXCUI as ID, r2.STR as DP, r1.RXCUI1 as Ingredient_ID
        FROM RXNREL r1
        JOIN RXNCONSO r2
        ON r1.RXCUI2 = r2.RXCUI
        WHERE ({s}) and r2.TTY = "DP"
    ),
    keys_all AS (
        SELECT ID
        FROM base
        GROUP by ID
        HAVING COUNT(DISTINCT Ingredient_ID) < {len(Ing_lst)}
    )
    SELECT b.ID,b.DP
    FROM base b
    JOIN keys_all k
    ON b.ID = k.ID
    WHERE b.Id != {ID}
    GROUP BY b.ID, b.DP
    """
    
    res = pd.read_sql(query, engine)
    
    lst = []
    drp = []
    for j,i in enumerate(res["DP"]): 
        if "[" in i:
            d = i.split("[")
            lst.append(d[-1][:-1])
        else:
            lst.append("Generic")
            drp.append(j)

    res["Product_Name"] = lst
    
    Product = []
    for j,i in enumerate(res["Product_Name"]):
        if i.lower() in Product:
            drp.append(j)
        else:
            Product.append(i.lower())
    res = res.drop(drp)
    res = res.drop_duplicates(subset="ID")
    res = res.reset_index(drop=True)
    return res
    
df = Union_Drugs(ingredients,1092378)
df

,ID,DP,Product_Name
0,198439,"ACETAMINOPHEN 500 mg ORAL CAPSULE, LIQUID FILL...",CVS Health Extra Strength Acetaminophen Softgels
1,198440,ACETAMINOPHEN 500 mg ORAL TABLET [Acetaminophe...,Acetaminophen ES
2,200977,"ACETAMINOPHEN 500 mg ORAL TABLET, FILM COATED ...",PANADOL Extra Strength
3,209443,ACETAMINOPHEN 500 mg ORAL TABLET [Extra Streng...,Extra Strength Mapap
4,209459,"ACETAMINOPHEN 500 mg ORAL TABLET, FILM COATED ...",Tylenol Extra Strength
5,209890,ACETAMINOPHEN 500 mg ORAL CAPSULE [MAPAP Extra...,MAPAP Extra Strength
6,247324,ACETAMINOPHEN 500 mg / DEXTROMETHORPHAN HYDROB...,Theraflu Flu Relief Max Daytime
7,307686,ACETAMINOPHEN 500 mg / CAFFEINE 65 mg ORAL TAB...,Tension Headache Aspirin Free
8,404172,ACETAMINOPHEN 500 mg / CAFFEINE 65 mg ORAL TAB...,Excedrin Tension Headache
9,901814,IBUPROFEN 200 mg / DIPHENHYDRAMINE HYDROCHLORI...,Ibuprofen PM


In [93]:
def Fetch_Ingredients(ID):
    query = f"""
    SELECT c.STR as Full_String
    FROM RXNCONSO c
    JOIN RXNREL r
        ON c.RXCUI = r.RXCUI2
    WHERE r.RXCUI1 = "{ID}"
      AND c.TTY = "SCDC"
    GROUP BY Full_String;
    """

    df = pd.read_sql(query, engine)

    import re
    results = []

    for text in df["Full_String"]:
        match = re.match(r"(.+?)\s+(\d+)\s+MG", text)
        if match:
            ingredient = match.group(1)
            concentration = float(match.group(2))
            results.append({
                "Ingredient": ingredient,
                "Concentration": concentration
            })

    return results

In [ ]:
import pandas as pd

def Fetch_Heatmap(df, drug_of_interest_id, drug_of_interest_name):

    # Add searched drug
    searched_row = pd.DataFrame({
        "ID": [drug_of_interest_id],
        "Product_Name": [drug_of_interest_name]
    })

    df_extended = pd.concat([searched_row, df], ignore_index=True)

    # Build long-format table
    rows = []

    for _, row in df_extended.iterrows():
        ingredients = Fetch_Ingredients(row["ID"])

        for ing in ingredients:
            rows.append({
                "Product_Name": row["Product_Name"],
                "Ingredient": ing["Ingredient"],
                "Concentration": ing["Concentration"]
            })

    long_df = pd.DataFrame(rows)

    # Pivot into matrix
    heatmap_df = long_df.pivot_table(
        index="Product_Name",
        columns="Ingredient",
        values="Concentration",
        fill_value=0
    ).reset_index()

    return heatmap_df


In [95]:
heatmap_df = Fetch_Heatmap(df,1092378,"Tylenol PM")
heatmap_df

Ingredient,Product_Name,acetaminophen,aspirin,caffeine,chlorpheniramine maleate,dexbrompheniramine maleate,dextromethorphan hydrobromide,diphenhydramine citrate,diphenhydramine hydrochloride,guaifenesin,ibuprofen,naproxen sodium,pamabrom,phenylephrine hydrochloride,pseudoephedrine hydrochloride,pyrilamine maleate
0,ACTIDOGESIC DF,500.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Acetaminophen ES,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aleve PM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,220.0,0.0,0.0,0.0,0.0
3,Alka-Seltzer Plus Day Severe Cold Plus Flu,500.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,400.0,0.0,0.0,0.0,10.0,0.0,0.0
4,Allergy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Allergy Relief,325.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
6,Assured Headache PM (Aspirin Free),500.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,BC MAX,500.0,500.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Banophen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Benadryl Allergy Liqui-Gels,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
import altair as alt
import pandas as pd

def Create_Altair_Heatmap(heatmap_df):

    df_long = heatmap_df.melt(
        id_vars=["Product_Name"],
        var_name="Ingredient",
        value_name="Concentration"
    )

    # Remove zero concentrations (optional but cleaner visually)
    df_long = df_long[df_long["Concentration"] > 0]

    # 🔹 Normalize concentration within each ingredient
    df_long["Relative_Conc"] = (
        df_long.groupby("Ingredient")["Concentration"]
        .transform(lambda x: x / x.max())
    )

    chart = alt.Chart(df_long).mark_rect().encode(
        x=alt.X('Ingredient:N', axis=alt.Axis(labelAngle=-45)),
        y='Product_Name:N',
        color=alt.Color(
            'Relative_Conc:Q',
            scale=alt.Scale(scheme='blues', domain=[0, 1]),
            title='Relative Concentration'
        ),
        tooltip=[
            'Product_Name',
            'Ingredient',
            alt.Tooltip('Concentration:Q', title='Concentration (mg)'),
            alt.Tooltip('Relative_Conc:Q', format='.2f')
        ]
    ).properties(
        width=700,
        height=400,
        title="Ingredient Concentration Heatmap (Relative per Ingredient)"
    )

    return chart

In [101]:
Create_Altair_Heatmap(heatmap_df)

alt.Chart(...)